In [1]:
# Import our SparkSession so we can use it
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.sql.functions import col

# Create our SparkSession, this can take a couple minutes locally
spark = SparkSession.builder.appName("BusinessJSON").config('spark.sql.broadcastTimeout',-1).getOrCreate()


In [2]:
df_business = spark.read.json("../data_source/business_review/part-00000-a6c5280a-4adf-4f1d-b37f-cbfd7dc9a6c6-c000.json")


In [39]:
df_business.printSchema()


root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [20]:
df_business.groupBy("business_id", "stars").count().show()


+--------------------+-----+-----+
|         business_id|stars|count|
+--------------------+-----+-----+
|-I06hkMFrX0KBqu61...|  5.0|    3|
|74LU6K2ro5AQXKT0J...|  4.0|   16|
|7EnzO970Zdrd_gjIJ...|  4.5|   19|
|93yfzr8B1s-mhan2W...|  3.0|    6|
|l3DOX58AX5mcHqnbJ...|  3.5|    6|
|xdBeEql6sCe7zBHZx...|  5.0|    7|
|FYR3nVpNvY2X6dBu1...|  5.0|    3|
|IVRpIYUnWpvSneeGH...|  5.0|   10|
|L7onn3bgx-8HgKERS...|  5.0|   12|
|dtp3UPOH3Fig5zW2t...|  3.5|    3|
|lcqzUoxA6E2aVIF7_...|  1.0|    3|
|UBOUVw45T86I6FgXA...|  5.0|    4|
|UcCJfq0PElINpflaT...|  2.5|   14|
|XetqQFNRm_lqfNZ6Y...|  2.5|    7|
|ZdF_p8ff2pNuZcX_A...|  3.5|    3|
|lgdde-A-rnzOw9ybm...|  3.5|   59|
|qzPW50oW2TN1yG8XK...|  3.5|    3|
|rDpX16jDGa4YGiycp...|  4.0|    3|
|z5QStGLj6ua31xXnN...|  5.0|    6|
|412zay6TFdL20X764...|  2.5|    6|
+--------------------+-----+-----+
only showing top 20 rows



In [36]:
df_business = df_business.orderBy(df_business.state.desc())


+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+----------+-----------+--------------------+-----------+---------+------------+--------------------+-----+-----+--------------------+--------------------+
|             address|          attributes|         business_id|          categories|      city|               hours|is_open|  latitude|  longitude|                name|postal_code|rev_stars|review_count|           review_id|stars|state|                text|             user_id|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+----------+-----------+--------------------+-----------+---------+------------+--------------------+-----+-----+--------------------+--------------------+
|     61 Queen Street|[,, 'none', {'rom...|FByZsT1Sob5Vf1AYJ...|Indian, Pakistani...|     Leeds|                null|      1|43.6528212|-79.3763454|         Des

In [45]:
States_Counting = df_business.groupBy("state").count()

States_Counting.orderBy("count").show(100)

+-----+-------+
|state|  count|
+-----+-------+
|   TN|      3|
|  DUR|      3|
|  CON|      3|
|   BC|      3|
|  XGL|      3|
|  DOW|      4|
|  BAS|      4|
|   UT|      4|
|   AK|      7|
|   AR|      7|
|   NJ|      8|
|   VT|      8|
|  XWY|      8|
|   AL|     12|
|   NE|     12|
|   GA|     14|
|  XGM|     14|
|   NM|     14|
|   VA|     16|
|   CT|     17|
|   WA|     19|
|   CA|    251|
|   NY|    283|
|   FL|    726|
|   TX|   1071|
|   SC|  21272|
|   IL|  42371|
|   AB|  99639|
|   WI| 133660|
|   QC| 179039|
|   PA| 290097|
|   OH| 321345|
|   NC| 408060|
|   ON| 784461|
|   AZ|2082951|
|   NV|2320491|
+-----+-------+



In [50]:
df_business.filter(df_business.state == "GA").groupBy("city").count().show(200)


+---------+-----+
|     city|count|
+---------+-----+
| Avondale|   11|
|Las Vegas|    3|
+---------+-----+



In [49]:
df_business.filter((col("rev_stars") >= 4 ) & (col("state") == 'NV') & (col("city") == 'North Las Vegas')).groupBy("business_id", "rev_stars").count().show()



+--------------------+---------+-----+
|         business_id|rev_stars|count|
+--------------------+---------+-----+
|0yYwrT1vYV0Jvzj74...|      5.0|    8|
|UbL2j7VYnz8deaPUx...|      5.0|    2|
|mBKkTiufDsw56kPmO...|      5.0|   17|
|sqWrGiCjR9wwVPc3J...|      4.0|    2|
|gGS9Os0UBBO8BosD3...|      5.0|   24|
|ripszTzwIaQxa0ySL...|      4.0|    1|
|qrSc13CYKlu5P94oj...|      5.0|    5|
|Xm50kPf3xFpO8OVO2...|      4.0|    4|
|BB9fhkkfc0K_pKVM6...|      4.0|    1|
|5ZW87XKCg_FUJkeI2...|      5.0|    4|
|8uU3DrUjqqNsIcrrU...|      5.0|    9|
|E0vt4bpkuR7vGloNH...|      4.0|    1|
|YxvYHkQjlesf7x9fa...|      5.0|    4|
|gmiNx6RjVBnjoWZy8...|      5.0|    4|
|okyjI_AzNv-qWcb2X...|      5.0|    1|
|NczmALAiqjAjKGJJM...|      5.0|   44|
|cfSuc4q7pQLF28mMb...|      5.0|   75|
|FkXpr1ms8toOZ4sUW...|      4.0|    2|
|RVKYE3WnFFT7yKIGm...|      4.0|    2|
|Ev4xCXADFtLSkeZbl...|      4.0|    3|
+--------------------+---------+-----+
only showing top 20 rows



In [55]:
df_business.select("categories").groupBy("categories").count().show(100, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                       |count|
+-------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Health & Medical, Dentists, Oral Surgeons, Doctors, Cosmetic Dentists                                                                            |20   |
|Music & DVDs, Videos & Video Game Rental, Vinyl Records, Books, Mags, Music & Video, Shopping                                                    |3    |
|Parks, Active Life, Dog Parks                                                                                                                    |252  |
|Breakfast & Brunch, Burgers, Sandwiches, Restaurants, American (Traditional

In [60]:
df_business.where(col("categories").like("%Auto%")).select("categories").groupBy("categories").count().show(100, False)



+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                                                             |count|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Shopping, Sporting Goods, Local Services, Automotive, Motorcycle Dealers, Bikes, Bike Repair/Maintenance                                                                               |6    |
|Automotive, Car Dealers, Auto Parts & Supplies, Auto Repair                                                                                                                            |1207 |
|Electronics, Shopping, Home Theatre Ins

In [66]:
df_business.select("attributes.RestaurantsPriceRange2").groupBy("RestaurantsPriceRange2").count().show(100, False)


+----------------------+-------+
|RestaurantsPriceRange2|count  |
+----------------------+-------+
|3                     |523610 |
|None                  |677    |
|null                  |1177506|
|1                     |1355845|
|4                     |100846 |
|2                     |3527416|
+----------------------+-------+



In [70]:
df_business.filter("attributes.RestaurantsPriceRange2 == 'None'").select("attributes.RestaurantsPriceRange2","categories").groupBy("categories").count().show(100, False)



+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                                                                    |count|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Nightlife, Dive Bars, Restaurants, American (Traditional), Bars                                                                                                                               |4    |
|Food, Ice Cream & Frozen Yogurt                                                                                                                                                               |5    |
|Beau